In [37]:
import tushare as ts
import pandas as pd
import time
import datetime
from tqdm import tqdm
import re
import pprint

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [3]:
companys = pro.stock_basic(exchange='', list_status='L', fields='ts_code')
com_list = list(companys['ts_code'])

In [13]:
today = datetime.date.today()
end = today.strftime('%Y%m%d')

In [14]:
holder_list = []
for code in tqdm(com_list):
    time_df = pro.top10_holders(ts_code=code, end_date=end)
    ann_date = time_df.iloc[0, 1]
    df = pro.top10_holders(ts_code=code, ann_date=ann_date)
    holder_list.append(df)
    time.sleep(0.5)

100%|██████████| 3829/3829 [46:22<00:00,  1.38it/s]


In [198]:
relation_df = pd.concat(holder_list)

In [199]:
len(relation_df)

53316

In [200]:
relation_df.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
0,000001.SZ,20200421,20200331,中国平安保险(集团)股份有限公司-集团本级-自有资金,9.618540e+09,49.56
1,000001.SZ,20200421,20200331,香港中央结算有限公司(陆股通),1.669875e+09,8.60
2,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-自有资金,1.186100e+09,6.11
3,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-传统-普通保险产品,4.404787e+08,2.27
4,000001.SZ,20200421,20200331,中国证券金融股份有限公司,4.292327e+08,2.21


In [201]:
relation_df[relation_df['holder_name'] == '天津中冀万泰投资管理有限公司']

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
10,300428.SZ,20200416,20200331,天津中冀万泰投资管理有限公司,2832861.0,0.49
26,300428.SZ,20200416,20191231,天津中冀万泰投资管理有限公司,2832861.0,0.49


In [202]:
relation_df = relation_df.drop_duplicates(['ts_code', 'ann_date', 'holder_name'])

In [203]:
len(relation_df)

39534

In [204]:
def filter_func(x: pd.Series) -> str:
    s = x['holder_name']
    s = s.split('-')[0]  # 去掉横杠
    s = s.split('一')[0]
    s = re.sub('\\(.*\\)', '', s)  # 去掉空格和内容
    s = re.sub('股份|责任|有限|公司|集团|省|市', '', s)
    return s

In [205]:
relation_df['name'] = relation_df.apply(filter_func, axis=1)

In [206]:
relation_df.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio,name
0,000001.SZ,20200421,20200331,中国平安保险(集团)股份有限公司-集团本级-自有资金,9.618540e+09,49.56,中国平安保险
1,000001.SZ,20200421,20200331,香港中央结算有限公司(陆股通),1.669875e+09,8.60,香港中央结算
2,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-自有资金,1.186100e+09,6.11,中国平安人寿保险
3,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-传统-普通保险产品,4.404787e+08,2.27,中国平安人寿保险
4,000001.SZ,20200421,20200331,中国证券金融股份有限公司,4.292327e+08,2.21,中国证券金融


In [207]:
len(set(relation_df['name']))

26865

In [208]:
len(set(relation_df['holder_name']))

28758

In [209]:
holder_df = relation_df['name'].to_frame()

In [210]:
holder_df = holder_df.drop_duplicates()
holder_df = holder_df.reset_index(drop=True)

In [211]:
len(holder_df)

26865

In [212]:
holder_df['holderId:ID(Holder)'] = holder_df.index

In [213]:
holder_df[holder_df['holderId:ID(Holder)'] == 7]

,name,holderId:ID(Holder)
7,全国社保基金,7


In [214]:
hold2id = {
    row['name']: row['holderId:ID(Holder)']
    for i, row in holder_df.iterrows()
}

In [215]:
relation_df[':START_ID(Holder)'] = relation_df.apply(lambda x: hold2id[x['name']], axis=1)

In [216]:
relation_df.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio,name,:START_ID(Holder)
0,000001.SZ,20200421,20200331,中国平安保险(集团)股份有限公司-集团本级-自有资金,9.618540e+09,49.56,中国平安保险,0
1,000001.SZ,20200421,20200331,香港中央结算有限公司(陆股通),1.669875e+09,8.60,香港中央结算,1
2,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-自有资金,1.186100e+09,6.11,中国平安人寿保险,2
3,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-传统-普通保险产品,4.404787e+08,2.27,中国平安人寿保险,2
4,000001.SZ,20200421,20200331,中国证券金融股份有限公司,4.292327e+08,2.21,中国证券金融,3


In [217]:
relation_df = relation_df.drop(columns=['end_date', 'hold_amount', 'name'])

In [218]:
relation_df.head()

,ts_code,ann_date,holder_name,hold_ratio,:START_ID(Holder)
0,000001.SZ,20200421,中国平安保险(集团)股份有限公司-集团本级-自有资金,49.56,0
1,000001.SZ,20200421,香港中央结算有限公司(陆股通),8.60,1
2,000001.SZ,20200421,中国平安人寿保险股份有限公司-自有资金,6.11,2
3,000001.SZ,20200421,中国平安人寿保险股份有限公司-传统-普通保险产品,2.27,2
4,000001.SZ,20200421,中国证券金融股份有限公司,2.21,3


In [219]:
relation_df['ann_date'] = pd.to_datetime(relation_df['ann_date'])

In [220]:
relation_df.columns = [':END_ID(Company)', 'annDate:date','holderFullname:string', 'holdRatio:double', ':START_ID(Holder)']

In [221]:
holder_df[':LABEL'] = 'Holder'
relation_df[':TYPE'] = 'top10_holder'

In [222]:
holder_df.head()

,name,holderId:ID(Holder),:LABEL
0,中国平安保险,0,Holder
1,香港中央结算,1,Holder
2,中国平安人寿保险,2,Holder
3,中国证券金融,3,Holder
4,中央汇金资产管理,4,Holder


In [223]:
relation_df.head()

,:END_ID(Company),annDate:date,holderFullname:string,holdRatio:double,:START_ID(Holder),:TYPE
0,000001.SZ,2020-04-21,中国平安保险(集团)股份有限公司-集团本级-自有资金,49.56,0,top10_holder
1,000001.SZ,2020-04-21,香港中央结算有限公司(陆股通),8.60,1,top10_holder
2,000001.SZ,2020-04-21,中国平安人寿保险股份有限公司-自有资金,6.11,2,top10_holder
3,000001.SZ,2020-04-21,中国平安人寿保险股份有限公司-传统-普通保险产品,2.27,2,top10_holder
4,000001.SZ,2020-04-21,中国证券金融股份有限公司,2.21,3,top10_holder


In [224]:
holder_df.to_csv('import/holder.csv', index=False)
relation_df.to_csv('import/holder_company.csv', index=False)